In [2]:
import pandas as pd
import numpy as np
import sys

In [48]:
## input is the netMPHpan xlsx for multiple sequence but single allel
## for single allel, load the xls file and filter by cutoff
data = pd.read_csv("text2.xls",sep="\t",skiprows=1)
df_epitope = data.loc[data['EL_Rank'] <2] 
df_epitope['core_tcrf'] = df_epitope.apply(lambda x:core_tcrf(x['core']),axis=1)
seq_epidist=non_conserved_d(df_epitope)
seq_epidist

/Users/jianichen1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,seq1,seq2,epitope_distance
0,KF826828-A-2004,KF530260-A-2005,0.0670
1,KF826828-A-2004,KF826838-A-2006,0.9483
2,KF826828-A-2004,KF826841-A-2007,0.0000
3,KF826828-A-2004,KY883567-A-2015,0.7337
4,KF530260-A-2005,KF826838-A-2006,1.0153
5,KF530260-A-2005,KF826841-A-2007,0.0670
6,KF530260-A-2005,KY883567-A-2015,0.8007
7,KF826838-A-2006,KF826841-A-2007,0.9483
8,KF826838-A-2006,KY883567-A-2015,0.0000
9,KF826841-A-2007,KY883567-A-2015,0.7337


In [34]:
def core_tcrf(core):
    core_tcrf = f'___{core[3]}{core[4]}{core[5]}{core[6]}{core[7]}_'
    return core_tcrf

In [35]:
## non-cross conserved  class I eptiopes determined by TCRf 4,5,6,7,8
def epitope_distance(core1,core2,score1,score2):
    if core1[3]==core2[3] and core1[4]==core2[4] and core1[5]==core2[5] and core1[6]==core2[6] and core1[7]==core2[7]:
        return 0
    else:
        return score1+score2

In [45]:
## example to calculate sequence distance with seq[1] and seq[2]
## i and are j are the index of seq list

def seq_dist(df_eptiope,i,j):
    seq1_epitope = df_eptiope.loc[df_eptiope['ID']==seq[i]]
    seq2_epitope = df_eptiope.loc[df_eptiope['ID']==seq[j]]
    epitope_compare = pd.merge(seq1_epitope, seq2_epitope, how="outer", on=["Pos","core_tcrf"])
    epitope_compare['EL-score_x']=epitope_compare['EL-score_x'].fillna(0)
    epitope_compare['EL-score_y']=epitope_compare['EL-score_y'].fillna(0)
    epitope_compare['core_x']=epitope_compare['core_x'].fillna('---------')
    epitope_compare['core_y']=epitope_compare['core_y'].fillna('---------')
    epitope_compare['epi_dist'] = epitope_compare.apply(lambda x: epitope_distance(str(x['core_x']),str(x['core_y']),x['EL-score_x'],x['EL-score_y']), axis =1)
    seq_distance = epitope_compare['epi_dist'].sum()
    return(seq_distance)


In [46]:
## loop through all seqs in the eitope prediction dataframe to get the no-conserved epitope distance

def non_conserved_d(df_epitope):
    seq = df_epitope.ID.unique()
    seq_distance=[]
    for i in range(len(seq)):
        for j in range(len(seq)):
            if i<j:
                distance = seq_dist(df_epitope,i,j)
                seq_distance.append([seq[i], seq[j], distance])
                df_distance = pd.DataFrame(seq_distance, columns=["seq1", "seq2","epitope_distance"])           
    return(df_distance)
